## Data cleaning

### Set up

We first import the different libraries that we will be using for this project

In [1]:
import pandas as pd
from datetime import datetime 


We import our dataset

In [2]:
data_path="data/fires-time-series.xlsx"
try:
    df=pd.read_excel(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
df.head()

,superficie,fecha,lat,lng,idprovincia,idmunicipio,municipio
0,20.0,1980-09-18,42.428281,-6.914337,32,9,"BARCO DE VALDEORRAS, O"
1,2.0,1983-01-16,42.542185,-8.449205,36,12,COTOBADE
2,6.0,1983-01-16,42.102572,-8.415920,36,34,"NEVES, AS"
3,3.0,1983-01-17,43.629834,-7.367642,27,19,FOZ
4,40.0,1983-01-18,43.018968,-7.408954,27,11,CASTROVERDE


### Group by weeks

Agrupamos los datos y obtenemos el número total de incendios y la superficie quemada por semana.

In [3]:
# Asegurarse de que la columna 'fecha' sea datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Agregar columna de semana (Año-Semana)
df['semana'] = df['fecha'].dt.to_period('W').apply(lambda r: r.start_time)

# Agrupar por semana
df = df.groupby('semana').agg(
    numero_incendios=('superficie', 'count'),  # Cuenta las filas
    superficie_total=('superficie', 'sum')    # Suma la superficie
).reset_index()

# Crear rango completo de semanas desde el primer lunes de 1983 hasta la última fecha
start_date = pd.Timestamp("1983-01-01")
end_date = df['semana'].max()

# Generar rango de semanas completas
full_weeks = pd.date_range(start=start_date, end=end_date, freq='W-MON')

# Crear un DataFrame con las semanas completas
weeks_df = pd.DataFrame({'semana': full_weeks})

# Combinar con el DataFrame original (left join)
df = weeks_df.merge(df, on='semana', how='left')

# Rellenar valores NaN con 0 para las columnas numéricas
df['numero_incendios'] = df['numero_incendios'].fillna(0).astype(int)
df['superficie_total'] = df['superficie_total'].fillna(0)

# Agregar columnas de año y número de semana
df['año'] = df['semana'].dt.year
df['semana_año'] = df['semana'].dt.isocalendar().week

df.head()

,semana,numero_incendios,superficie_total,año,semana_año
0,1983-01-03,0,0.0,1983,1
1,1983-01-10,2,8.0,1983,2
2,1983-01-17,61,937.0,1983,3
3,1983-01-24,75,1009.3,1983,4
4,1983-01-31,2,6.0,1983,5


### Split train and test data

Los datos para entrenamiento son el 80% inicial (1983-2011) y los de test el 20% restante (2012-2018).

In [4]:
# Dividir los datos en train y test
df_train = df[df['año'] <= 2011].reset_index(drop=True)
df_test = df[df['año'] >= 2012].reset_index(drop=True)

# Calcular tamaños de train y test
train_size = len(df_train)
test_size = len(df_test)
total_size = len(df)

# Calcular porcentajes
train_percentage = (train_size / total_size) * 100
test_percentage = (test_size / total_size) * 100

# Mostrar resultados
print(f"\nTamaño total: {total_size}")
print(f"Train: {train_size} filas ({train_percentage:.2f}%)")
print(f"Test: {test_size} filas ({test_percentage:.2f}%)")

# Verificar los resultados
print("Train:")
print(df_train.head(), "\n--------------\n", df_train.tail())
print("\nTest:")
print(df_test.head(), "\n--------------\n", df_test.tail()) # No se usará hasta el final


Tamaño total: 1879
Train: 1513 filas (80.52%)
Test: 366 filas (19.48%)
Train:
      semana  numero_incendios  superficie_total   año  semana_año
0 1983-01-03                 0               0.0  1983           1
1 1983-01-10                 2               8.0  1983           2
2 1983-01-17                61             937.0  1983           3
3 1983-01-24                75            1009.3  1983           4
4 1983-01-31                 2               6.0  1983           5 
--------------
          semana  numero_incendios  superficie_total   año  semana_año
1508 2011-11-28                 3             48.55  2011          48
1509 2011-12-05                 0              0.00  2011          49
1510 2011-12-12                 0              0.00  2011          50
1511 2011-12-19                 0              0.00  2011          51
1512 2011-12-26                 5             12.89  2011          52

Test:
      semana  numero_incendios  superficie_total   año  semana_año
0 2012-0

## Train models

### Analyze train data set

In [5]:
#TODO

### Time series

Tendremos que hacer 3 modelos con series temporales:
- Predecir numero de incendios
- Predecir hectareas quemadas
- Predecir ambas cosas a la vez

Después introducir también variables exógenas (meteorológicas).

In [6]:
#TODO

## Performances

In [7]:
#TODO

## Test Model


In [8]:
#TODO